In [21]:
import numpy as np
from random import seed
from random import randrange
from csv import reader

In [22]:
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

The function gini_index is defined
below are examples to run
# Examples

In [23]:
gini= gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1])
print(gini)
gini= gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1])
print(gini)

0.5
0.0


# Creating split

In [24]:
def test_split(index, value, dataset):
    left, right= list(), list()
    for row in dataset:
        if row[index]< value:
            left.append(row)
        else:
            right.append(row)
    return left,right

In [25]:
def get_split(dataset):
    class_values= list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups= 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups= test_split(index, row[index], dataset)
            gini= gini_index(groups, class_values)
            if gini<b_score:
                b_index, b_value, b_score, b_groups= index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

The variables b_index, b_value, b_score, b_groups are the best ones we need 

Below is a dummy dataset to check the function get_split()

In [26]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
split= get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 6.642]


# Building tree

Building a tree may be divided into 3 main parts:

    1.Terminal Nodes.
    2.Recursive Splitting.
    3.Building a Tree.


Below is a function named to_terminal() that will select a class value for a group of rows.

It returns the most common output value in a list of rows.

In [27]:
def to_terminal(group):
    outcomes=[row[-1] for row in group]
    return max(set(outcomes),key= outcomes.count)

Building a decision tree involves calling the above developed get_split() function over and over again on the groups created for each node.

In [28]:
def split(node, max_depth, min_size, depth):
    left,right= node['groups']
    del(node['groups'])
    if not left or not right:
        node['left']= node['right']= to_terminal( left+right )
        return
    if depth >= max_depth:
        node['left'], node['right']= to_terminal(left), to_terminal(right)
        return
    if len(left)<= min_size:
        node['left']= to_terminal(left)
    else:
        node['left']= get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right']= to_terminal(right)
    else:
        node['right']= get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Building a tree

In [29]:
def build_tree(train, max_depth, min_size):
    root= get_split(train)
    split(root, max_depth, min_size, 1)
    return root

Here is a print_tree function that can be used to print the tree that we have created in order to get a better understanding of the tree.

In [30]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

An example dataset

In [31]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
tree= build_tree(dataset, 1, 1)
print_tree(tree)

[X1 < 6.642]
 [0]
 [1]


Defining the predict function

In [32]:
def predict(node, row):
    if row[node['index']]< node['value']:
        if isinstance(node['left'],dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'],dict):
            return predict(node['right'], row)
        else:
            return node['right']

Here is a testing of our prediction

In [33]:
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
stump= {'index':0, 'right':1, 'value':6.642287351, 'left':0}
for row in dataset:
    prediction= predict(stump, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


# Working on a dataset

In [34]:
def load_csv(filename):
    file= open(filename, "r")
    lines= reader(file)
    dataset= list(lines)
    return dataset

In [35]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column]= float(row[column].strip())

In [36]:
def cross_validation(dataset, n_folds):
    dataset_split= list()
    dataset_copy= list(dataset)
    fold_size= int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold= list()
        while len(fold)< fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [37]:
def accuracy_metric(actual, predicted):
    correct=0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct+=1
    return correct/ float(len(actual)) * 100.0

In [38]:
def evaluate_algorithm(dataset, algorithm, n_fold, *args):
    folds= cross_validation(dataset, n_folds)
    scores= list()
    for fold in folds:
        train_set= list(folds)
        train_set.remove(fold)
        train_set= sum(train_set, [])
        test_set= list()
        for row in fold:
            row_copy= list(row)
            test_set.append(row_copy)
            row_copy[-1]= None
        predicted= algorithm(train_set, test_set, *args)
        actual= [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [39]:
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [40]:
seed(1)
filename= 'data_banknote_authentication.csv'
dataset= load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
n_folds, max_depth, min_size = 5,5,10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print("Scores::"+ str(scores))
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores::[96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%


The values for n_folds, max_depth, min_size can be changed accordingly